In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
%matplotlib inline

In [5]:
data = pd.read_csv('/Users/logno/Documents/GitHub/diabetes-project/diabetes.csv')
df = data.copy()

# Broad overview of the data

In [6]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [7]:
df.shape

(768, 9)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [9]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [10]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Cleaning

Despite there being no N/A values in this dataset, I believe that there are some values in certain columns that are incorrect. For example, it is impossible for a person to have an insulin value of zero, and if they did, that would be an issue. In this section, I will clean up these values and ensure that all of the data makes sense.

## Finding which columns to check

In [11]:
df.loc[df['Glucose'] == 0].shape

(5, 9)

In [12]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [13]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [14]:
df.loc[df['Age'] == 0].shape

(0, 9)

In [15]:
df.loc[df['DiabetesPedigreeFunction'] == 0].shape

(0, 9)

In [16]:
bad_insulin = df.loc[df['Insulin'] == 0]
ins = bad_insulin[['Insulin', 'Outcome']]
ins['Outcome'].sum()

138

It seems that the columns with the missing values are glucose, bloodpressure, and BMI. 

## Glucose

In [17]:
# Finding the rows that have a glucose of 0
df.loc[df['Glucose'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


Since there are only 5 rows, I think that it would be best to drop them.

In [18]:
# Dropping the problem rows and creating a new dataframe
df = df.drop(df.index[[75,182,342,349,502]])

In [19]:
# Checking if the values were dropped
df.loc[df['Glucose'] == 0].shape

(0, 9)

## Blood Pressure

In [20]:
df['BloodPressure'].describe()

count    763.000000
mean      69.115334
std       19.399240
min        0.000000
25%       62.000000
50%       72.000000
75%       80.000000
max      122.000000
Name: BloodPressure, dtype: float64

In [21]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [22]:
# Finding how many null values there are
df['BloodPressure'] = df['BloodPressure'].replace({0: np.nan})
df['BloodPressure'].isnull().sum()

35

Since there are a fair number of null values, I will impute them using the median.

In [23]:
# Imputing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')

df["BloodPressure"] = imp.fit_transform(df[["BloodPressure"]]).ravel()
df['BloodPressure'].isnull().sum()

0

## BMI

In [24]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [25]:
df['BMI'].describe()

count    763.000000
mean      31.986763
std        7.893221
min        0.000000
25%       27.300000
50%       32.000000
75%       36.550000
max       67.100000
Name: BMI, dtype: float64

I will impute these values as well, just for fun. I can come back to this later.

In [26]:
# Imputing values
df["BMI"] = imp.fit_transform(df[["BMI"]]).ravel()
df['BMI'].isnull().sum()

0

Time to double check that I am done cleaning.

In [27]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [28]:
df.shape

(763, 9)

# Building models

In [29]:
# Creating my splits
from sklearn.model_selection import train_test_split

y = df.Outcome.values
X = df.drop(['Outcome'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=27)

## RandomForestClassifier

In [30]:
# Building the base model
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(random_state=27)
rfm.fit(X_train, y_train.ravel())

RandomForestClassifier(random_state=27)

In [31]:
# Finding the model's predictions
predictions = rfm.predict(X_test)

from sklearn import metrics

print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, predictions)))

Accuracy = 0.778


I got an accuracy of 78%, which is not bad. I will test some other models and compare.

## XGBoostClassifier

In [35]:
import xgboost as xgb
xgb_classif = xgb.XGBClassifier()
xgb_classif = xgb_classif.fit(X_train, y_train)

print('The accuracy of the xgboost classifier is {:.2f} out of 1 on the training data'.format(xgb_classif.score(X_train, y_train)))
print('The accuracy of the xgboost classifier is {:.2f} out of 1 on the test data'.format(xgb_classif.score(X_test, y_test)))

The accuracy of the xgboost classifier is 1.00 out of 1 on the training data
The accuracy of the xgboost classifier is 0.75 out of 1 on the test data


XGBoost gave me an accuracy of 75%, not too bad either.

## KNN Classifier

In [36]:
from sklearn.neighbors import KNeighborsClassifier


test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [37]:
## score that comes from testing on the same datapoints that were used for training
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

Max train score 100.0 % and k = [1]


In [38]:
## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

Max test score 76.47058823529412 % and k = [14]


In [40]:
#Setup a knn classifier with k neighbors
knn = KNeighborsClassifier(14)

knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.7647058823529411

In [42]:
#import confusion_matrix
from sklearn.metrics import confusion_matrix
#let us get the predictions using the classifier we had fit above
y_pred = knn.predict(X_test)
confusion_matrix(y_test,y_pred)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,87,9,96
1,27,30,57
All,114,39,153


In [43]:
# Classification_report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

precision    recall  f1-score   support

           0       0.76      0.91      0.83        96
           1       0.77      0.53      0.62        57

    accuracy                           0.76       153
   macro avg       0.77      0.72      0.73       153
weighted avg       0.77      0.76      0.75       153



The KNN Classifier got an accuracy of 76% with mostly true positives, which is good.